<a href="https://colab.research.google.com/github/hanbuck30/neural-layer/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [305]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [306]:
timesteps = 5 # 시점의 수. NLP에서는 보통 문장의 길이
input_size = 3 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량
output_size=5
batch_size=1
inputs = torch.randn(batch_size,timesteps,input_size)
hidden_state_t=torch.zeros(hidden_size,requires_grad=True)
num_layers=2


In [307]:
Wx = torch.randn(hidden_size, input_size,requires_grad=True) 
Wh = torch.randn(hidden_size, hidden_size,requires_grad=True)
b = torch.randn(hidden_size,requires_grad=True)
learning_rate = 0.1

In [308]:
print(Wx.shape)
print(Wh.shape)
print(b.shape)

torch.Size([8, 3])
torch.Size([8, 8])
torch.Size([8])


In [309]:
inputs.shape

torch.Size([1, 5, 3])

In [310]:
Y=torch.LongTensor([[0,1,2,3,4]])
y_data=[Y]
Y.shape

torch.Size([1, 5])

In [311]:
b_fc=torch.randn(output_size,requires_grad=True)
w_fc=torch.randn(hidden_size, output_size,requires_grad=True)

In [314]:
total_hidden_state=[]
total_hidden_states=[]
def RNNmodel(inputs,hidden_state_t,num_layers):
    def RNN(inputs,hidden_state_t,num_layers):
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                total_hidden_states.append(output_t) 
                hidden_state_t = output_t
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def RNN_two(inputs,hidden_state_t,num_layers):
        for k in range(1,num_layers):
            globals()['W{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            globals()['b{}'.format(k)]=torch.randn(hidden_size,requires_grad=True)
            globals()['Wh{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                hidden_state_t = output_t
                output_t1 = torch.tanh(torch.matmul(W1,hidden_state_t) + torch.matmul(Wh1,hidden_state_t1) + b1)
                hiddent_state_t1=output_t1
                total_hidden_states.append(output_t1)
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def RNN_multi(inputs,hidden_state_t,num_layers):
        for k in range(1,num_layers):
            globals()['W{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            globals()['b{}'.format(k)]=torch.randn(hidden_size,requires_grad=True)
            globals()['Wh{}'.format(k)]=torch.randn(hidden_size,hidden_size,requires_grad=True)
            
        for i in inputs:
            for input_t in i: 
                output_t = torch.tanh(torch.matmul(Wx,input_t) + torch.matmul(Wh,hidden_state_t) + b)
                hidden_state_t = output_t
                output_t1 = torch.tanh(torch.matmul(W1,hidden_state_t) + torch.matmul(Wh1,hidden_state_t1) + b1)
                hiddent_state_t1=output_t1
                for m in range(2,num_layers):
                    globals()['output_t{}'.format(m)] = torch.tanh(torch.matmul(globals()['W{}'.format(m)],globals()['hidden_state_t{}'.format(m)]) + torch.matmul(globals()['Wh{}'.format(m)],globals()['hidden_state_t{}'.format(m)]) + globals()['b{}'.format(m)])
                    globals()['hidden_state_t{}'.format(m)] = globals()['output_t{}'.format(m)]
                total_hidden_states.append(globals()['output_t{}'.format(num_layers-1)])
            total_state = torch.stack(total_hidden_states,dim=0)
            total_hidden_state.append(total_state)
        total_state=torch.stack(total_hidden_state,dim=0)
        return total_state
    def fc(total_state):
        linear=torch.matmul(total_state,w_fc)+b_fc
        return linear
    if num_layers == 1:
        v=fc(RNN(inputs,hidden_state_t,num_layers))
        return v
    elif num_layers== 2 :
        v=fc(RNN_two(inputs,hidden_state_t,num_layers))
        return v
    else:
        v=fc(RNN_multi(inputs,hidden_state_t,num_layers))
        return v
RNNmodel(inputs,hidden_state_t,3)

tensor([[[ 3.0253,  2.0219, -0.1979,  0.2563,  2.1793],
         [-0.9591,  5.2763,  4.1475,  7.7717, -1.0933],
         [ 0.6286,  0.5239,  2.2686,  1.4442,  4.6062],
         [ 1.9244, -1.0175, -3.4309, -1.9974,  1.3570],
         [ 4.1051,  3.2610,  0.7107,  0.9630,  2.9602]]],
       grad_fn=<AddBackward0>)

In [315]:
parameters=[Wx,Wh,b,b_fc,w_fc]
def make_parameters(num_layers):
    for k in range(1,num_layers):
            parameters.append(globals()['W{}'.format(k)])
            parameters.append(globals()['b{}'.format(k)])
            parameters.append(globals()['Wh{}'.format(k)])
            parameters.append(globals()['hidden_state_t{}'.format(k)])
make_parameters(num_layers)


In [316]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(parameters, learning_rate)

In [317]:
for i in range(100):
    total_hidden_state=[]
    total_hidden_states=[]
    optimizer.zero_grad()
    outputs=RNNmodel(inputs,hidden_state_t,num_layers)
    loss = criterion(outputs.view(-1,output_size), Y.view(-1)) # view를 하는 이유는 Batch 차원 제거를 위해
    loss.backward() 
    optimizer.step() 

    
    result = outputs.data.numpy().argmax(axis=2) 
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data)

0 loss:  2.5676021575927734 prediction:  [[2 1 1 4 2]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
1 loss:  1.328209638595581 prediction:  [[4 0 1 4 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
2 loss:  3.414651393890381 prediction:  [[4 3 1 0 3]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
3 loss:  2.8375041484832764 prediction:  [[4 0 3 1 1]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
4 loss:  3.963907241821289 prediction:  [[3 1 1 1 3]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
5 loss:  2.9066600799560547 prediction:  [[4 0 0 3 1]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
6 loss:  3.0715537071228027 prediction:  [[3 3 0 1 1]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
7 loss:  1.9304507970809937 prediction:  [[1 1 1 1 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
8 loss:  2.9184157848358154 prediction:  [[3 0 0 4 1]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
9 loss:  2.603450298309326 prediction:  [[3 3 3 4 4]] true Y:  [tensor([[0, 1, 2, 3, 4]])]
10 loss:  2.7019143104553223 prediction:  [[3 2 4 3 3]] true Y:  [tensor([[0, 1, 2, 